This notebook is taken directly from https://github.com/tcapelle/llm_recipes/tree/main

# From Llama to Alpaca: Finetunning and LLM with Weights & Biases


In [1]:
# !pip install wandb transformers trl datasets "protobuf==3.20.3" evaluate

## With Huggingface TRL

Let's grab the Alpaca (GPT-4 curated instructions and outputs) dataset:

In [2]:
# !wget https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json
from utilities.parse_benchmark import parse_benchmark

benchmark = "MedQA"
benchmark_questions, benchmark_answers = parse_benchmark(benchmark)
# print(benchmark_questions[0])
# print(benchmark_answers[0])

Loading Benchmark from MedQA-USMLE/US/test.jsonl
Benchmark contains 1273 questions, made up of 1273 with 5 options and 0 with non-5 options


In [3]:
import wandb
wandb.init(project="biollama_ft", # the project I am working on
           tags=["hf_sft", "BioLlama"]) # the Hyperparameters I want to keep track of
# artifact = wandb.use_artifact('Neelectric/MedQA-USMLE', type='dataset')
# artifact_dir = artifact.download()

wandb: Currently logged in as: nelectric (neelectric). Use `wandb login --relogin` to force relogin


wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.16.1


wandb: Run data is saved locally in /home/service/BioLlama/wandb/run-20240301_183746-jalopya9
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run ruby-haze-153


wandb: ⭐️ View project at https://wandb.ai/neelectric/biollama_ft


wandb: 🚀 View run at https://wandb.ai/neelectric/biollama_ft/runs/jalopya9


In [4]:
import os
# print(artifact_dir)
artifact_dir = os.getcwd() + "/benchmarks/MedQA-USMLE/"
from datasets import load_dataset
#dataset = load_dataset("Neelectric/MedQA-USMLE")
medqa = load_dataset("json", data_dir=artifact_dir)

/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#trying gsutil for SciFive pretraining corpus
# !pip install gsutil
import pandas as pd
import numpy as np
abs_1_16 = pd.read_csv("abs_1_16.tsv", sep='\t')
abs_1_30 = pd.read_csv("abs_1_30.tsv", sep='\t')
# combine these two and then convert to a dataset
abs_combined = pd.concat([abs_1_16, abs_1_30])
print(abs_combined)
# use load_dataset to convert to a dataset
from datasets import Dataset
abs_combined = Dataset.from_pandas(abs_combined)
print(abs_combined)

        Unnamed: 0  \
0              NaN   
1              NaN   
2              NaN   
3              NaN   
4              NaN   
...            ...   
473153         NaN   
473154         NaN   
473155         NaN   
473156         NaN   
473157         NaN   

       LAUGHING GAS is the newest thing for kids seeking kicks, the Stanford Daily reports. "They sniff it."So begins a news story in the Los Angeles Times of 26 January 1967. The story continues:"It's the latest way to travel, or so say a growing group of devotees on the campus," the university student paper said. "It can produce much the same effects as psychedelic drugs, they claim, and it's cheaper to obtain.""One student said he buys the gas, nitrous oxide, from a medical supply house. ;They think I am anesthetizing rats,' he explained."Campus medical authorities said the gas, sniffed ;in sufficient amounts... could produce all the states of anesthesia, including the final stage-death.'"  \
0       The role of angiograph

Dataset({
    features: ['Unnamed: 0', 'LAUGHING GAS is the newest thing for kids seeking kicks, the Stanford Daily reports. "They sniff it."So begins a news story in the Los Angeles Times of 26 January 1967. The story continues:"It\'s the latest way to travel, or so say a growing group of devotees on the campus," the university student paper said. "It can produce much the same effects as psychedelic drugs, they claim, and it\'s cheaper to obtain.""One student said he buys the gas, nitrous oxide, from a medical supply house. ;They think I am anesthetizing rats,\' he explained."Campus medical authorities said the gas, sniffed ;in sufficient amounts... could produce all the states of anesthesia, including the final stage-death.\'"', 'To determine the extent to which the Brunnstrom recovery stages of upper limb in hemiparetic stroke patients are correlated to neurophysiological measures and the spasticity measure of Modified Modified Ashworth Scale (MMAS).', '__index_level_0__'],
    num_

In [6]:
medqa

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
        num_rows: 10178
    })
    validation: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
        num_rows: 1272
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
        num_rows: 1273
    })
})

Let's log the dataset also as a table so we can inspect it on the workspace.

In [7]:
train_dataset = medqa["train"]
eval_dataset = medqa["validation"]
#print sizes
print(len(train_dataset))
print(len(eval_dataset))
# turn both of these into only half their size
# train_dataset = train_dataset.select(range(0, len(train_dataset)//2))
# eval_dataset = eval_dataset.select(range(0, len(eval_dataset)//2))

# print(len(train_dataset))
# print(len(eval_dataset))

10178
1272


In [8]:
def create_prompt(row):
    option_string = ""
    for option in row["options"].keys():
        option_string += "\n (" + option + ") " + row["options"][option]
    row["option_string"] = option_string
    return ("<QUESTION>{question} {option_string}</QUESTION><ANSWER> ({answer_idx}) {answer}</ANSWER>").format_map(row)
create_prompt(train_dataset[4])

"<QUESTION>A 20-year-old woman presents with menorrhagia for the past several years. She says that her menses “have always been heavy”, and she has experienced easy bruising for as long as she can remember. Family history is significant for her mother, who had similar problems with bruising easily. The patient's vital signs include: heart rate 98/min, respiratory rate 14/min, temperature 36.1°C (96.9°F), and blood pressure 110/87 mm Hg. Physical examination is unremarkable. Laboratory tests show the following: platelet count 200,000/mm3, PT 12 seconds, and PTT 43 seconds. Which of the following is the most likely cause of this patient’s symptoms? \n (A) Factor V Leiden\n (B) Hemophilia A\n (C) Lupus anticoagulant\n (D) Protein C deficiency\n (E) Von Willebrand disease</QUESTION><ANSWER> (E) Von Willebrand disease</ANSWER>"

In [9]:
def create_prompt_no_answer(row):
    option_string = ""
    for option in row["options"].keys():
        option_string += "\n (" + option + ") " + row["options"][option]
    row["option_string"] = option_string
    return ("<QUESTION>{question} {option_string}</QUESTION><ANSWER> ").format_map(row)

def return_prompt_no_answer(row):
    return {"text": create_prompt_no_answer(row)}

def return_prompt(row):
    return {"text": create_prompt(row)}
    
test_dataset = eval_dataset.map(return_prompt_no_answer)
# print(test_dataset[0]["text"])
train_dataset_with_texts = train_dataset.map(return_prompt)
# print(train_dataset_with_texts[0]["text"])

Training the full models is expensive, but if you have a GPU that can fit the full model, you can skip this part. Let's just train the last 8 layers of the model (Llama2-7B has 32)

In [10]:
size = "7"
if size == "7":
    RETRO_layer_ids = [15]
elif size == "13":
    RETRO_layer_ids = [19]
elif size == "70":
    RETRO_layer_ids = [39]

In [11]:
from utilities.biollama import BioLlama
import torch

# questions = ["Which is the main calcium pump of the sarcoplasmic reticulum? Answer:"]
amended_questions = ["The main calcium pump of the sarcoplasmic reticulum is "]
questions = amended_questions
# answers = ["Sarcoplasmic reticulum Ca(2+)-ATPase"] # or "SERCA","serca2"

prompt = questions[0]
# model_id = "TheBloke/Llama-2-7b-chat-GPTQ"
model_id = "meta-llama/Llama-2-" + size +"b-chat-hf"
chunk_length = 32

BioLlama = BioLlama(
    model_id=model_id,
    chunk_length=chunk_length,
    RETRO_layer_ids=RETRO_layer_ids,
    training=True,
    torch_dtype=torch.float32,
)

Loading checkpoint shards:   0%|                                                            | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:  50%|██████████████████████████                          | 1/2 [00:03<00:03,  3.72s/it]

Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.49s/it]

Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.67s/it]

/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [12]:
# print(BioLlama.model)
model = BioLlama.model
tokenizer = BioLlama.tokenizer

In [13]:
print("freezing layers, currently only works for single unfrozen retro layer")
n_freeze = BioLlama.RETRO_layer_ids[0]
# n_freeze = 15

# freeze layers (disable gradients)
for param in model.parameters(): 
    param.requires_grad = False
for param in model.lm_head.parameters(): 
    param.requires_grad = True
#for every parameter in retro_layer_params, print where in the model it comes from (ie is it from self attention, layer norm, etc)
print(f"\nprinting layer {n_freeze} params")
for name, param in model.model.layers[n_freeze].named_parameters():
    print(f"{name}, requires_grad = {param.requires_grad}")   

list_of_params_to_unfreeze = [
    "cca_attn.q_proj.weight",
    "cca_attn.k_proj.weight",
    "cca_attn.v_proj.weight",
    "cca_attn.o_proj.weight",
    "pre_cca_layernorm.weight",
]

for name, param in model.model.layers[n_freeze].named_parameters(): 
    if name in list_of_params_to_unfreeze:
        param.requires_grad = True
print("\nprinting layer 15 params")
for name, param in model.model.layers[n_freeze].named_parameters():
    print(f"{name}, requires_grad = {param.requires_grad}")   

freezing layers, currently only works for single unfrozen retro layer

printing layer 15 params
self_attn.q_proj.weight, requires_grad = False
self_attn.k_proj.weight, requires_grad = False
self_attn.v_proj.weight, requires_grad = False
self_attn.o_proj.weight, requires_grad = False
mlp.gate_proj.weight, requires_grad = False
mlp.up_proj.weight, requires_grad = False
mlp.down_proj.weight, requires_grad = False
input_layernorm.weight, requires_grad = False
post_attention_layernorm.weight, requires_grad = False
cca_attn.q_proj.weight, requires_grad = False
cca_attn.k_proj.weight, requires_grad = False
cca_attn.v_proj.weight, requires_grad = False
cca_attn.o_proj.weight, requires_grad = False
pre_cca_layernorm.weight, requires_grad = False

printing layer 15 params
self_attn.q_proj.weight, requires_grad = False
self_attn.k_proj.weight, requires_grad = False
self_attn.v_proj.weight, requires_grad = False
self_attn.o_proj.weight, requires_grad = False
mlp.gate_proj.weight, requires_grad = F

In [14]:
for name, param in model.model.named_parameters(): 
    # param.requires_grad = True
    #print if needs grad
    print(f"{name}, requires_grad = {param.requires_grad}")

BioLlama.model.train()

embed_tokens.weight, requires_grad = False
layers.0.self_attn.q_proj.weight, requires_grad = False
layers.0.self_attn.k_proj.weight, requires_grad = False
layers.0.self_attn.v_proj.weight, requires_grad = False
layers.0.self_attn.o_proj.weight, requires_grad = False
layers.0.mlp.gate_proj.weight, requires_grad = False
layers.0.mlp.up_proj.weight, requires_grad = False
layers.0.mlp.down_proj.weight, requires_grad = False
layers.0.input_layernorm.weight, requires_grad = False
layers.0.post_attention_layernorm.weight, requires_grad = False
layers.1.self_attn.q_proj.weight, requires_grad = False
layers.1.self_attn.k_proj.weight, requires_grad = False
layers.1.self_attn.v_proj.weight, requires_grad = False
layers.1.self_attn.o_proj.weight, requires_grad = False
layers.1.mlp.gate_proj.weight, requires_grad = False
layers.1.mlp.up_proj.weight, requires_grad = False
layers.1.mlp.down_proj.weight, requires_grad = False
layers.1.input_layernorm.weight, requires_grad = False
layers.1.post_attenti

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-14): 15 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (15): LlamaDecoderLayer(
        (self_attn

In [15]:
# Just freeze embeddings for small memory decrease
# model.model.embed_tokens.weight.requires_grad_(False);

In [16]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

params, trainable_params = param_count(model)

Total params: 6805.53M, Trainable: 198.18M


In [17]:
batch_size = 2

total_num_steps = 11_210 // batch_size
print(total_num_steps)


total_num_steps = 5000
print(f"changing total num size to {total_num_steps}")

5605
changing total num size to 5000


In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments
output_dir = "/home/service/BioLlama/utilities/finetuning/biollama_training_output/" + size  + "/"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    bf16=True,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=total_num_steps // 10,
    num_train_epochs=10,
    max_steps=total_num_steps,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    eval_steps=5000,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=8,
    save_strategy="epoch", #changed to epoch so we save every epoch i guess?
    save_total_limit=1,
)

In [19]:
# from utils import LLMSampleCB, token_accuracy
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset_with_texts,
    dataset_text_field="text",
    eval_dataset=test_dataset,
    packing=True,
    max_seq_length=1024,
    args=training_args,
    formatting_func=create_prompt,
    # compute_metrics=token_accuracy,
)

/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(
/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [20]:
#very hacky but maybe this will work:
tokenizer.model_input_names = ['labels', 'input_ids', 'attention_mask']
# trainer.args.train_batch_size = 1
# self.args.train_batch_size

#also hacky, but could work:
tokenizer.pad_token = tokenizer.eos_token
print("Starting training")
trainer.train()
wandb.finish()

Starting training


/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


In [ ]:
size = "7"
output_dir = "/home/service/BioLlama/utilities/finetuning/biollama_training_output/" + size  + "/"
RETRO_layer_ids = [15]

In [ ]:
import os
print(os.path.abspath(output_dir))
trainer.save_model(output_dir)
# !ls -l $output_dir

In [ ]:
size = "7"
output_dir = "/home/service/BioLlama/utilities/finetuning/biollama_training_output/" + size  + "/"
RETRO_layer_ids = [15]

In [ ]:
#load this local model here and use it to generate some text
print(output_dir)

from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import torch
from utilities.biollama import BioLlama

chunk_length = 32

BioLlama = BioLlama(model_id=output_dir, 
    chunk_length=chunk_length, 
    RETRO_layer_ids = RETRO_layer_ids, 
    training=False, 
    torch_dtype=torch.float32)

In [ ]:
prompt  = '<QUESTION>A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient? \n (A) Ampicillin\n (B) Ceftriaxone\n (C) Ciprofloxacin\n (D) Doxycycline\n (E) Nitrofurantoin</QUESTION>\n<ANSWER> '
time_before_generation = time.time()
num_tokens, text = BioLlama.generate(prompt=prompt, max_new_tokens=50)
time_after = time.time()

print("***Generating***")
print(text)
print(f"Time taken for generation: {time_after - time_before_generation}")
print(f"Tokens per second: {num_tokens/(time_after - time_before_generation)}")

In [ ]:
prompt2 = '<QUESTION>A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby? \n (A) Placing the infant in a supine position on a firm mattress while sleeping\n (B) Routine postnatal electrocardiogram (ECG)\n (C) Keeping the infant covered and maintaining a high room temperature\n (D) Application of a device to maintain the sleeping position\n (E) Avoiding pacifier use during sleep</QUESTION>\n<ANSWER> '
time_before_generation = time.time()
num_tokens, text = BioLlama.generate(prompt=prompt2, max_new_tokens=50)
time_after = time.time()

print("***Generating***")
print(text)
print(f"Time taken for generation: {time_after - time_before_generation}")
print(f"Tokens per second: {num_tokens/(time_after - time_before_generation)}")

In [ ]:
prompt3 = "<QUESTION>A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complications and has not had any medical problems up until this time. However, for the past 4 days, he has been fussy, is regurgitating all of his feeds, and his vomit is yellow in color. On physical exam, the child's abdomen is minimally distended but no other abnormalities are appreciated. Which of the following embryologic errors could account for this presentation? \n (A) Abnormal migration of ventral pancreatic bud\n (B) Complete failure of proximal duodenum to recanalize\n (C) Error in neural crest cell migration\n (D) Abnormal hypertrophy of the pylorus\n (E) Failure of lateral body folds to move ventrally and fuse in the midline</QUESTION>\n<ANSWER> "
time_before_generation = time.time()
num_tokens, text = BioLlama.generate(prompt=prompt3, max_new_tokens=50)
time_after = time.time()

print("***Generating***")
print(text)
print(f"Time taken for generation: {time_after - time_before_generation}")
print(f"Tokens per second: {num_tokens/(time_after - time_before_generation)}")

In [ ]:
prompt4 = "<QUESTION>A 20-year-old woman presents with menorrhagia for the past several years. She says that her menses “have always been heavy”, and she has experienced easy bruising for as long as she can remember. Family history is significant for her mother, who had similar problems with bruising easily. The patient's vital signs include: heart rate 98/min, respiratory rate 14/min, temperature 36.1°C (96.9°F), and blood pressure 110/87 mm Hg. Physical examination is unremarkable. Laboratory tests show the following: platelet count 200,000/mm3, PT 12 seconds, and PTT 43 seconds. Which of the following is the most likely cause of this patient’s symptoms? \n (A) Factor V Leiden\n (B) Hemophilia A\n (C) Lupus anticoagulant\n (D) Protein C deficiency\n (E) Von Willebrand disease</QUESTION>\n<ANSWER> "
time_before_generation = time.time()
num_tokens, text = BioLlama.generate(prompt=prompt4, max_new_tokens=50)
time_after = time.time()

print("***Generating***")
print(text)
print(f"Time taken for generation: {time_after - time_before_generation}")
print(f"Tokens per second: {num_tokens/(time_after - time_before_generation)}")